In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
with open('dataset_book.txt') as file:
    data=file.read()

In [3]:
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer

#     non_html=re.sub('<.*?>',' ',x)   #remove html tag using regular expresion
soup=BeautifulSoup(data,'html.parser')#remove html tag using beautifulsoup
non_html_text=soup.get_text()

#remove unwanted charectors and symbols
text=re.sub('[^a-zA-Z0-9\s]',' ',non_html_text)
text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')
#remove extra spaces
z = []
for i in text.split():
    if i not in z:
        z.append(i)
text = ' '.join(z)

#tokenize text

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
sequence=tokenizer.texts_to_sequences([text])[0]

sequence[:15]

[39, 1, 40, 424, 425, 426, 2, 427, 428, 3, 429, 4, 41, 430, 42]

In [4]:
ip_dim=len(tokenizer.word_index)+1
ip_dim

6746

In [5]:
#traning sequence

sequences = []

for i in range(3, len(sequence)):
    words = sequence[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)


x,y=[],[]
for i in sequences:
    x.append(i[0:3])
    y.append(i[3])
x=np.array(x)
y=np.array(y)

The Length of sequences are:  7203


In [6]:
x[:10],y[:10]

(array([[ 39,   1,  40],
        [  1,  40, 424],
        [ 40, 424, 425],
        [424, 425, 426],
        [425, 426,   2],
        [426,   2, 427],
        [  2, 427, 428],
        [427, 428,   3],
        [428,   3, 429],
        [  3, 429,   4]]),
 array([424, 425, 426,   2, 427, 428,   3, 429,   4,  41]))

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# # # Pad "sequences" to a fixed length
# max_len = 10
# x = pad_sequences(x, maxlen=max_len)
# y = pad_sequences(y, maxlen=max_len)

# Convert y to categorical

y = to_categorical(y, num_classes=ip_dim)
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
model=Sequential()
model.add(Embedding(ip_dim,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(ip_dim,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             67460     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 6746)              6752746   
                                                                 
Total params: 19869206 (75.80 MB)
Trainable params: 19869206 (75.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
# It gives single predicted words
def predict_nxt_word(txt):
#

    tok_text = tokenizer.texts_to_sequences([ txt])
#     print(tok_text)
    preds = model.predict(np.array(tok_text), verbose=0)[0]
    next_word = tokenizer.sequences_to_texts([[np.argmax(preds)]])[0]
    return next_word

In [13]:
# It gives 5 predicted words

def predict_nxt_words1(txt, k=5):
    tok_text = tokenizer.texts_to_sequences([txt])
    preds = model.predict(np.array(tok_text), verbose=0)[0]
    top_k_indices = preds.argsort()[-k:][::-1]
    top_k_words = [tokenizer.sequences_to_texts([[i]])[0] for i in top_k_indices]
    return top_k_words